In [4]:
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Dependencies
import pandas as pd
import pdb

# Import BeautifulSoup
from bs4 import BeautifulSoup as bs
import requests

# Import Splinter and set the chromedriver path
from splinter import Browser
from selenium import webdriver

# Module used to connect Python with MongoDb
import pymongo

In [5]:
data_file = "covid_mex_21apr.csv"
data_df = pd.read_csv(data_file)
data_df.columns = data_df.columns.str.replace(' ', '_')
data_df.columns = data_df.columns.str.lower()
data_df

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,...,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-04-21,09e8dc,2,9,15,1,15,15,37,2,...,2,2,2,2,2,1,99,99,97,1
1,2020-04-21,1dd782,2,12,9,1,15,9,3,1,...,2,1,2,2,2,2,99,99,97,97
2,2020-04-21,0efbaf,2,9,28,2,16,28,32,1,...,2,2,2,2,1,1,99,99,97,97
3,2020-04-21,013a6c,99,3,15,2,15,15,106,1,...,2,1,2,2,1,3,99,99,97,97
4,2020-04-21,091a48,1,12,15,2,15,15,31,2,...,2,2,2,2,2,2,99,99,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51629,2020-04-21,071ee2,2,12,27,1,27,27,4,1,...,2,2,2,2,1,2,99,99,97,97
51630,2020-04-21,0cc924,1,4,30,1,20,30,39,1,...,2,2,2,2,99,2,99,México,97,97
51631,2020-04-21,145db8,1,4,30,2,30,30,26,1,...,2,2,2,2,99,2,99,México,97,97
51632,2020-04-21,17eb11,1,12,21,1,21,21,3,1,...,2,2,2,2,2,2,99,99,97,97


In [6]:
columns_file = 'columns_meaning.csv'
columns_df = pd.read_csv(columns_file)
columns_df

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1.0,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2.0,ID_REGISTRO,Número identificador del caso,TEXTO
2,3.0,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4.0,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5.0,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6.0,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7.0,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8.0,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9.0,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10.0,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [7]:
rows_file = 'rows_meaning.csv'
rows_df = pd.read_csv(rows_file)
rows_df

,CLAVE_origen,DESCRIPCION_origen,CLAVE_secto,DESCRIPC_sector,CLAVE_sex,DESCRIPC_sex,CLAVE_tipo_pte,DESCRIPC_tipo_pte,CLAVE_si_no,DESCRIPC_si_no,CLAVE_nacionalidad,DESCRIPC_nacionalidad,CLAVE_resultado,DESCRIPC_resultado,CLAVE_ENTIDAD,desc_entidad,abreviatura_entidad,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_entidad_del_municipio
0,1.0,USMER,1.0,CRUZ ROJA,1.0,MUJER,1.0,AMBULATORIO,1.0,SI,1.0,MEXICANA,1.0,Positivo SARS-CoV-2,1.0,AGUASCALIENTES,AS,1,AGUASCALIENTES,1
1,2.0,FUERA DE USMER,2.0,DIF,2.0,HOMBRE,2.0,HOSPITALIZADO,2.0,NO,2.0,EXTRANJERA,2.0,No positivo SARS-CoV-2,2.0,BAJA CALIFORNIA,BC,2,ASIENTOS,1
2,99.0,NO ESPECIFICADO,3.0,ESTATAL,99.0,NO ESPECIFICADO,99.0,NO ESPECIFICADO,97.0,NO APLICA,99.0,NO ESPECIFICADO,3.0,Resultado pendiente,3.0,BAJA CALIFORNIA SUR,BS,3,CALVILLO,1
3,NaN,NaN,4.0,IMSS,NaN,NaN,NaN,NaN,98.0,SE IGNORA,NaN,NaN,NaN,NaN,4.0,CAMPECHE,CC,4,COSÍO,1
4,NaN,NaN,5.0,IMSS-BIENESTAR,NaN,NaN,NaN,NaN,99.0,NO ESPECIFICADO,NaN,NaN,NaN,NaN,5.0,COAHUILA DE ZARAGOZA,CL,5,JESÚS MARÍA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,NO ESPECIFICADO,32
2497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,NO ESPECIFICADO,36
2498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,997,NO APLICA,97
2499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,998,SE IGNORA,98


Scrape the Website and collect the information needed

In [ ]:
# Scrape the Web site (datos.gob.mx) and collect the latest reports 
# Assign the text to variables that you can reference later.

# URL of page to be scraped
url_gob = "https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico"

# executable path and browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
browser.visit(url_gob)

# Retrieve page with the requested module
response = requests.get(url_gob)

In [ ]:
# Create Beatifulsoup object; pase with 'html.parser' or 'lxml'
soup = bs(browser.html, 'html.parser')

# Examine the results, then determine element that contains the info needed
#print(soup.prettify()) #Note: it was very useful to print the first time, afterwards it is im